<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/3_make_errors/e3_make_errors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

Mounted at /content/drive


**아이디어 기록**

1. Critical

2. Major
    - Mis-Translation
        - "ailen" -> 외계인으로 처리

# 0.Data Load

In [2]:
import pandas as pd
import numpy as np
import json
import os
import re
import copy
from tqdm.notebook import tqdm

if os.path.exists(r"G:\내 드라이브\KB인턴관련"):
    gdrive_path = r"G:\내 드라이브\KB인턴관련"
else:
    gdrive_path = "/content/drive/MyDrive/KB인턴관련"

In [3]:
df = pd.read_csv(os.path.join(gdrive_path, "preprocessing/fin.csv"), encoding="euc-kr", index_col=0)
df.head()

,category,lawName,ENG,KOR
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발·운영의 지원 및 개성공업지구에 투자하거나 출입·체류하는...
2,헌법,개성공업지구 지원에 관한 법률,"The term ""Gaeseong Industrial Complex"" or ""GIC...","""개성공업지구""란 남한과 북한 사이의 합의에 따라서 북한의 개성지역 일대에서 개발·..."
3,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC developer"" means a South Korean ...","""개성공업지구 개발업자""란 「남북교류협력에 관한 법률」에 따른 협력사업의 승인을 받..."
4,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC management organ"" means a corpor...","""개성공업지구 관리기관""이란 개성공업지구의 관리·운영을 위하여 북한의 「개성공업지구..."
5,헌법,개성공업지구 지원에 관한 법률,"The term ""local enterprise of the GIC"" means a...","""개성공업지구 현지기업""이란 남한주민이 「남북교류협력에 관한 법률」에 따른 협력사업..."


# 1.Data Fiiltering

In [4]:
keyword_idx = df.loc[df.lawName.str.contains(r"은행|금융|보험|예금|적금|대출|외환|통화|화폐|환율|예대")].index

In [5]:
tgk_idx = df.loc[df.category=="통화ㆍ국채ㆍ금융"].index

In [6]:
print(f"키워드: {len(keyword_idx)}\t통화/국채/금융: {len(tgk_idx)}")
print(f"키워드-통국금: {len(list(set(keyword_idx)-set(tgk_idx)))}")
print(f"통국금-키워드: {len(list(set(tgk_idx)-set(keyword_idx)))}")
print(f"전체:  {len(list(set(keyword_idx).union(set(tgk_idx))))}")

키워드: 16712	통화/국채/금융: 16004
키워드-통국금: 4681
통국금-키워드: 3973
전체:  20685


In [7]:
df_samp = df.loc[list(set(keyword_idx).union(set(tgk_idx)))].drop(index=[403163], axis=0)
df_samp

,category,lawName,ENG,KOR
547121,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the average monthly remuneration of each...,"예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술..."
547123,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the amount of monthly remuneration of ea...,"노무제공자[「고용보험법」 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""..."
547124,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,"""Where any ground prescribed by Presidential D...","법 제3조제1항에서 ""대통령령으로 정하는 사유에 해당하는 경우""란 다음 각 호의 어..."
547125,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,The standard remuneration under Article 3 (2) ...,법 제3조제2항(법 제48조의2제8항제2호 및 제48조의3제8항제2호에서 준용하는 ...
547126,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where an artist falls under any of the followi...,예술인이 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의2제8항제2호에서 준...
...,...,...,...,...
78005,통화ㆍ국채ㆍ금융,국채법,Where necessary to properly issue and manage S...,기획재정부장관은 국채의 발행과 관리를 적절하게 하기 위하여 필요한 경우에는 관계 중...
78006,통화ㆍ국채ㆍ금융,국채법,Any person requested to provide materials rela...,제1항에 따라 국채 관련 자료의 제출을 요청받은 자는 특별한 사유가 없으면 요청에 ...
78007,통화ㆍ국채ㆍ금융,국채법,Where necessary to formulate and carry out pol...,기획재정부장관은 국채시장에 관한 정책의 수립·운영에 필요한 경우에는 관계 중앙행정기...
406361,통화ㆍ국채ㆍ금융,신용협동조합법 시행령,"Notwithstanding paragraph (3), a workplace cre...",제3항의 규정에 불구하고 제12조제1항제2호의 직장조합 및 동항제3호 라목의 단체조...


In [8]:
df_samp["curHyp"] = df_samp.KOR.values
df_samp["errors"] = [[] for _ in range(df_samp.shape[0])]
df_samp["tagLog"] = [dict() for _ in range(df_samp.shape[0])]
# df_samp.iloc[:, 2:4]

**주요 함수**

In [15]:
## 주요한 함수
def remake_df():
    result = df.loc[list(set(keyword_idx).union(set(tgk_idx)))]
    result["curHyp"] = result.KOR.values
    result["errors"] = [[] for _ in range(result.shape[0])]
    result["tag_log"] = [dict() for _ in range(result.shape[0])]
    return result.drop(index=[403163], axis=0)

def random_df(dfs, ratio, seed):
    np.random.seed(seed)
    return dfs.loc[np.random.choice(dfs.index, int(dfs.shape[0]*ratio), False), :]

def random_index(srs, ratio, seed):
    np.random.seed(seed)
    return np.random.choice(srs.index, int(srs.shape[0]*ratio), False)

def copy_df(data, col=["errors", "tagLog"]):
    result = data.drop(columns=col, axis=1).copy(deep=True)
    # print(id(result), id(data))
    for c in col:
        result[c] = data[c].apply(lambda x: copy.deepcopy(x)).copy(deep=True)
    return result

def count_tag(tokens):
    token_dict = dict()
    for token in tokens:
        try:
            token_dict[token.tag].append(token.form)
        except:
            token_dict[token.tag] = [token.form]
    return token_dict

def find_nnp_tokens(tokens):
    nnp_tokens = []
    for token in tokens:
        if token.tag == "NNP":
            nnp_tokens.append((token.form, token.start, token.end))
    return nnp_tokens

def print_df(df, idx=0, end=0):
    print(df.shape[0])
    if end==0:
        values = df.index[idx:]
    else:
        values = df.index[idx:end]
    for i in values:
        print(f"_________________ {i} _________________")
        print(df.loc[i, "curHyp"])
        print(df.loc[i, "errors"])
        print(df.loc[i, "tagLog"])

# result.loc[i, "errors"].append({"severity":"major", "start_idx":start, "text":del_unit, "type":"omission"})
def input_errors(df, index, severity, start_idx, text, type):
    df.loc[index, "errors"].append({"severity":severity, "start_idx":start_idx, "text":text, "type":type})

def convert_df(df, diff_df):
    cdf = copy_df(df)
    for i in diff_df.index:
        cdf.loc[i, : ] = diff_df.loc[i, : ]
    return cdf

# 2.품사 태깅

In [11]:
from kiwipiepy import Kiwi

kiwi = Kiwi(num_workers=0, model_type='sbg')
kiwi_tokens = df_samp.KOR.apply(lambda x: kiwi.tokenize(x))
kiwi_tokens

547121    [(예술인, NNG, 0, 3), ([, SSO, 3, 1), (「, SSO, 4,...
547123    [(노무, NNG, 0, 2), (제공자, NNG, 2, 3), ([, SSO, 5...
547124    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547125    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547126    [(예술, NNG, 0, 2), (이, VCP, 2, 1), (ᆫ, ETM, 2, ...
                                ...                        
78005     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
78006     [(제, XPN, 0, 1), (1, SN, 1, 1), (항, NNG, 2, 1)...
78007     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
406361    [(제, XPN, 0, 1), (3, SN, 1, 1), (항, NNG, 2, 1)...
406362    [(조합, NNG, 0, 2), (이, JKS, 2, 1), (제, XPN, 4, ...
Name: KOR, Length: 20684, dtype: object

In [12]:
count_tokens = kiwi_tokens.apply(count_tag)
count_tokens

547121    {'NNG': ['예술인', '항', '단서', '단기', '예술인', '이하', ...
547123    {'NNG': ['노무', '제공자', '항', '단서', '단기', '노무', '...
547124    {'NNG': ['법', '항', '대통령령', '사유', '해당', '경우', '...
547125    {'NNG': ['법', '항', '법', '항', '항', '준용', '경우', ...
547126    {'NNG': ['예술', '다음', '호의', '해당', '경우', '법', '항...
                                ...                        
78005     {'NNG': ['기획', '재정부', '장관', '국채', '발행', '관리', ...
78006     {'XPN': ['제'], 'SN': ['1'], 'NNG': ['항', '국채',...
78007     {'NNG': ['기획', '재정부', '장관', '국채', '시장', '정책', ...
406361    {'XPN': ['제', '제', '제', '제', '제'], 'SN': ['3',...
406362    {'NNG': ['조합', '항', '규정', '복지', '사업', '실시', '경...
Name: KOR, Length: 20684, dtype: object

In [13]:
nng_dict = dict()
for tokens in count_tokens.apply(lambda x: x["NNG"]):
    for t in tokens:
        try:
            nng_dict[t] += 1
        except:
            nng_dict[t] = 1
len(nng_dict.keys())

3703

In [14]:
nnp_info = kiwi_tokens.apply(find_nnp_tokens)
nnp_info

547121    [(고용보험법, 5, 10)]
547123    [(고용보험법, 7, 12)]
547124                  []
547125                  []
547126                  []
                ...       
78005     [(한국은행, 56, 60)]
78006                   []
78007                   []
406361                  []
406362                  []
Name: KOR, Length: 20684, dtype: object

# 3.Error 생성

## 3.1.Critical

### 3.1.1.Critical - Omission

In [16]:
cr_om_index = random_index(df_samp.curHyp, 0.025, 10)
# print_df(df_samp.loc[cr_om_index])

In [17]:
def critical_omission(df):
    result = copy_df(df)
    random_ratio = [3/5, 2/3, 5/8, 5/7]
    np.random.seed(10)
    split_sentences = result.curHyp.apply(lambda x: x.split(" "))
    len_element = split_sentences.apply(len)
    crop_element = len_element.apply(lambda x: int(x*(np.random.choice(random_ratio, 1))))
    start_idx = (len_element-crop_element).apply(lambda x: np.random.choice([i for i in range(x)], 1)[0])
    end_idx = start_idx+crop_element

    for i in df.index:
        reduce_sentence = " ".join([split_sentences[i][k] for k in range(start_idx[i], end_idx[i]+1)])
        result.loc[i, "tagLog"]["omission"] = reduce_sentence
        cur_hyp = result.loc[i, "curHyp"]
        sidx = cur_hyp.find(reduce_sentence)

        conv_hyp = " ".join([split_sentences[i][j] for j in range(len_element[i]) if (start_idx[i] > j) or (end_idx[i] < j)])
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "critical", sidx, reduce_sentence, "omission")

    del split_sentences, len_element, crop_element, start_idx, end_idx
    return result
df_crom = convert_df(df_samp, critical_omission(df_samp.loc[cr_om_index]))

In [18]:
df_crom.loc[cr_om_index]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
85600,통화ㆍ국채ㆍ금융,풍수해보험법,Where the insured transfers an insured subject...,피보험자가 보험에 가입된 보험목적물을 양도하는 경우에는 그 양수인은 보험계약에 관한...,피보험자가 보험에 승계한 것으로 추정한다.,"[{'severity': 'critical', 'start_idx': 10, 'te...",{'omission': '가입된 보험목적물을 양도하는 경우에는 그 양수인은 보험계약...
411047,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률,The corporation (including its affiliated comp...,그 법인(그 계열회사를 포함한다. 이하 이 호 및 제2호에서 같다) 및 그 법인의 ...,직무와 관련하여 미공개중요정보를 알게 된 자,"[{'severity': 'critical', 'start_idx': 0, 'tex...",{'omission': '그 법인(그 계열회사를 포함한다. 이하 이 호 및 제2호에...
403214,통화ㆍ국채ㆍ금융,상호저축은행법 시행령,"""Matters prescribed by Presidential Decree"" in...","법 제2조제9호다목에서 ""대통령령으로 정하는 사항""이란 다음 각 호의 어느 하나에 ...",어느 하나에 해당하는 사항을 말한다.,"[{'severity': 'critical', 'start_idx': 0, 'tex...","{'omission': '법 제2조제9호다목에서 ""대통령령으로 정하는 사항""이란 다..."
82056,통화ㆍ국채ㆍ금융,서민의 금융생활 지원에 관한 법률,A welfare business entity (hereinafter referre...,이 법 시행 당시 「휴면예금관리재단의 설립 등에 관한 법률」에 따른 복지사업자(이하...,이 법 시행 당시 「휴면예금관리재단의,"[{'severity': 'critical', 'start_idx': 21, 'te...","{'omission': '설립 등에 관한 법률」에 따른 복지사업자(이하 ""복지사업자..."
521525,사회복지,노인장기요양보험법,A person who operates a long-term care institu...,장기요양기관을 운영하는 자는 제33조의2제3항의 영상정보가 분실·도난·유출·변조 또...,장기요양기관을 바에 따라 안전성 확보에 필요한 기술적·관리적·물리적 조치를 하여야 한다.,"[{'severity': 'critical', 'start_idx': 8, 'tex...",{'omission': '운영하는 자는 제33조의2제3항의 영상정보가 분실·도난·유...
...,...,...,...,...,...,...,...
555784,노동,산업재해보상보험법,"Matters necessary for the standards, methods, ...",제1항에 따른 간병급여의 지급 기준과 지급 방법 등에 관하여 필요한 사항은 대통령령...,제1항에 대통령령으로 정한다.,"[{'severity': 'critical', 'start_idx': 5, 'tex...",{'omission': '따른 간병급여의 지급 기준과 지급 방법 등에 관하여 필요한...
85000,통화ㆍ국채ㆍ금융,전자단기사채등의 발행 및 유통에 관한 법률,Where the amount under paragraph (1) exceeds t...,계좌관리기관은 제1호의 금액이 제2호의 금액을 초과하는 경우에는 대통령령으로 정하는...,계좌관리기관은 그 초과분을 말소하여야 한다.,"[{'severity': 'critical', 'start_idx': 8, 'tex...",{'omission': '제1호의 금액이 제2호의 금액을 초과하는 경우에는 대통령령...
83326,통화ㆍ국채ㆍ금융,온라인투자연계금융업 및 이용자 보호에 관한 법률,Every online investment-linked financial busin...,온라인투자연계금융업자는 이용자가 온라인투자연계금융업자의 영업건전성 및 온라인투자연계...,호의 정보를 자신의 온라인플랫폼을 통하여 공시하여야 한다.,"[{'severity': 'critical', 'start_idx': 0, 'tex...",{'omission': '온라인투자연계금융업자는 이용자가 온라인투자연계금융업자의 영...
81641,통화ㆍ국채ㆍ금융,대부업 등의 등록 및 금융이용자 보호에 관한 법률,If a credit service provider whose largest sha...,이 법 시행 당시 제10조제7항의 개정규정에 적합하지 아니한 여신금융기관이 최대주주...,이 법 시행 당시 하여야 한다.,"[{'severity': 'critical', 'start_idx': 10, 'te...",{'omission': '제10조제7항의 개정규정에 적합하지 아니한 여신금융기관이 ...


### 3.1.2.Mis-Translation

In [39]:
cr_mt_index = random_index(df_crom, 0.025, 111)
temp = df_crom.loc[cr_mt_index]
temp.loc[temp.errors.apply(len)>0].iloc[:, 3:]

,KOR,curHyp,errors,tagLog
403951,신용정보회사는 다음 각 호의 업무 외에는 총리령으로 정하는 바에 따라 금융위원회에 ...,신용정보회사는 다음 각 호의 법률에 따라 인가·허가·등록 및 승인 등을 미리 받아야...,"[{'severity': 'critical', 'start_idx': 16, 'te...",{'omission': '업무 외에는 총리령으로 정하는 바에 따라 금융위원회에 미리...
414454,투자자가 여행·질병 등으로 일시적으로 부재하는 중에 금융투자상품의 가격 폭락 등 불...,투자자가 여행·질병 등으로 금융투자상품의 매도 권한을 일임받은 경우,"[{'severity': 'critical', 'start_idx': 15, 'te...",{'omission': '일시적으로 부재하는 중에 금융투자상품의 가격 폭락 등 불가...
253728,제28조 및 제29조의 규정에 따라 허가를 받거나 등록을 하고자 하는 자는 다음 각...,제28조 및 제29조의 규정에 따라 허가를 받거나 한한다.,"[{'severity': 'critical', 'start_idx': 28, 'te...",{'omission': '등록을 하고자 하는 자는 다음 각 호의 요건을 모두 갖추어...
518657,"분쟁조정위원회는 위원장을 포함하여 60명 이내의 위원으로 구성하고, 위원장을 제외한...","분쟁조정위원회는 위원장을 포함하여 60명 이내의 위원으로 구성하고, 하여야 한다.","[{'severity': 'critical', 'start_idx': 38, 'te...",{'omission': '위원장을 제외한 위원 중 1명은 당연직위원으로 한다. 이 ...
397165,"금융지주회사등이 추가로 신용공여를 하지 아니하였음에도 불구하고 자기자본의 변동, 동...",금융지주회사등이 추가로 신용공여를 한도를 초과하게 되는 경우,"[{'severity': 'critical', 'start_idx': 19, 'te...","{'omission': '하지 아니하였음에도 불구하고 자기자본의 변동, 동일차주의 ..."
409917,"투자자문업자가 아닌 자는 그 상호 중에 ""투자자문""이라는 문자 또는 이와 같은 의미...",투자자문업자가 아닌 자는 그 상호 중에 대통령령으로 정하는 문자를 사용할 수 있다.,"[{'severity': 'critical', 'start_idx': 22, 'te...","{'omission': '""투자자문""이라는 문자 또는 이와 같은 의미를 가지는 외국..."
397179,은행지주회사등은 해당 은행지주회사의 주요출자자의 다른 회사에 대한 출자를 지원하기 ...,위한 신용공여를 하여서는 아니 된다.,"[{'severity': 'critical', 'start_idx': 0, 'tex...",{'omission': '은행지주회사등은 해당 은행지주회사의 주요출자자의 다른 회사...
519502,공표심의위원회 위원(제2항제5호의 위원은 제외한다)의 임기는 2년으로 한다.,공표심의위원회 위원(제2항제5호의,"[{'severity': 'critical', 'start_idx': 19, 'te...",{'omission': '위원은 제외한다)의 임기는 2년으로 한다.'}
82115,제7조부터 제10조까지의 규정에 따라 등기할 사항 중 금융위원회의 인가 또는 승인을...,제7조부터 해당 인가서 또는 승인서가 도달한 날부터 등기기간을 기산한다.,"[{'severity': 'critical', 'start_idx': 6, 'tex...",{'omission': '제10조까지의 규정에 따라 등기할 사항 중 금융위원회의 인...


In [ ]:
df_crom

## 3.1.Major - Omission (고유명사 생략)

In [ ]:
change_val = nnp_info.loc[nnp_info.apply(len) > 0].apply(lambda x: sorted(x, key=lambda e: len(e[0]), reverse=True)[0])
change_val

547121        (고용보험법, 5, 10)
547123        (고용보험법, 7, 12)
394199       (특별자치시, 65, 70)
394206          (관보, 65, 67)
394213        (경제자유구역, 0, 6)
                 ...        
416408      (주주총회, 105, 109)
194518    (산업재해보상보험, 25, 33)
78003          (한국은행, 8, 12)
78004           (한국은행, 0, 4)
78005         (한국은행, 56, 60)
Name: KOR, Length: 3039, dtype: object

In [ ]:
ch6 = change_val[change_val.apply(lambda x: len(x[0])) > 6]
drop_idx = ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1].index.to_list()
ch6

395122          (한국자산관리공사, 19, 27)
395133          (한국자산관리공사, 11, 19)
395168          (한국자산관리공사, 12, 20)
395180           (공적자금관리위원회, 0, 9)
395208          (한국자산관리공사, 12, 20)
                    ...           
194388    (산업재해보상보험재심사위원회, 12, 26)
194420           (산업재해보상보험, 9, 17)
194425           (산업재해보상보험, 8, 16)
194426           (산업재해보상보험, 8, 16)
194518          (산업재해보상보험, 25, 33)
Name: KOR, Length: 503, dtype: object

In [ ]:
ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1]    # 시리즈에 검출되는 고유명사가 있다면 drop 시킬 것!

Series([], Name: KOR, dtype: object)

In [ ]:
def critical_omission(ch_srs, df_local):
    result = copy_df(df_local)
    for i in ch_srs.index:
        del_word, start, end = ch_srs.loc[i]
        cur_hyp = result.loc[i, "curHyp"]
        forward = cur_hyp[:start]
        backward = cur_hyp[start:]
        split_back = backward.split(" ")
        del_unit = split_back[0]
        back_mod = " ".join(split_back[1:])
        try:
            if cur_hyp[end]== "(": continue
            elif start==0:
                result.loc[i, "curHyp"] = (forward+back_mod).strip(" ")
                result.loc[i, "errors"].append({"severity":"major", "start_idx":0, "text":del_unit, "type":"omission"})
            elif cur_hyp[start-1] == '"' or cur_hyp[start-1]=='「' :
                continue
            else:
                result.loc[i, "curHyp"] = (forward+back_mod).strip(" ")
                result.loc[i, "errors"].append({"severity":"major", "start_idx":start, "text":del_unit, "type":"omission"})
        except:
            pass

    return result

In [ ]:
# d_index= random_index(temp, 0.1)
co_case = critical_omission(ch3.drop(index=[403163]), df_samp)

In [ ]:
ch3.loc[547121]

('고용보험법', 5, 10)

In [ ]:
df_samp.loc[394372, "KOR"]

'경제자유구역(경제자유구역개발계획이 수립된 지역만 해당한다)에 있는 국가 또는 지방자치단체 소유의 토지로서 개발사업에 필요한 토지는 개발사업 외의 목적으로 매각하거나 양도할 수 없다.'

In [ ]:
df_samp.loc[co_case.errors.apply(len)>0]

,category,lawName,ENG,KOR,curHyp,errors
394199,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,A Mayor/Do Governor may request the Minister o...,"시·도지사는 산업통상자원부장관에게 경제자유구역의 지정을 요청할 수 있다. 다만, 대...","시·도지사는 산업통상자원부장관에게 경제자유구역의 지정을 요청할 수 있다. 다만, 대...","[{'severity': 'major', 'start_idx': 65, 'text'..."
394213,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,Feasibility of securing sites and infrastructu...,경제자유구역의 개발에 필요한 부지와 광역교통망·정보통신망·용수·전력 등 기반시설의 ...,개발에 필요한 부지와 광역교통망·정보통신망·용수·전력 등 기반시설의 확보가 가능할 것,"[{'severity': 'major', 'start_idx': 0, 'text':..."
394231,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,Matters concerning the supply of exclusive lan...,"외국인투자기업, 국내복귀기업, 첨단기술 및 첨단제품 투자 기업, 핵심전략산업 투자 ...","외국인투자기업, 국내복귀기업, 첨단기술 및 첨단제품 투자 기업, 핵심전략산업 투자 ...","[{'severity': 'major', 'start_idx': 120, 'text..."
394249,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,When any free economic zone is designated or m...,경제자유구역의 지정 또는 변경이 있은 때에는 그 경제자유구역개발계획의 내용에 따라 ...,지정 또는 변경이 있은 때에는 그 경제자유구역개발계획의 내용에 따라 다음 각 호의 ...,"[{'severity': 'major', 'start_idx': 0, 'text':..."
394310,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,Where a free economic zone designated pursuant...,산업통상자원부장관은 제4조에 따라 지정된 경제자유구역이 다음 각 호의 어느 하나에 ...,산업통상자원부장관은 제4조에 따라 지정된 다음 각 호의 어느 하나에 해당하는 경우에...,"[{'severity': 'major', 'start_idx': 23, 'text'..."
...,...,...,...,...,...,...
416408,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률 시행령,Where a person performs an act falling under a...,신문·방송·잡지 등 불특정 다수인에 대한 광고를 통하여 법 제152조제2항 각 호의...,신문·방송·잡지 등 불특정 다수인에 대한 광고를 통하여 법 제152조제2항 각 호의...,"[{'severity': 'major', 'start_idx': 105, 'text..."
194518,노동,산업재해보상보험법 시행령,The Industrial Accident Compensation Insurance...,"이 영 시행 당시 종전의 규정에 따라 설치된 산업재해보상보험심의위원회, 정책전문위원...",이 영 시행 당시 종전의 규정에 따라 설치된 정책전문위원회 및 요양전문위원회는 각각...,"[{'severity': 'major', 'start_idx': 25, 'text'..."
78003,통화ㆍ국채ㆍ금융,국채법,Article 36 (4) and (5) of the Management of th...,제1항에 따라 한국은행이 처리하는 국채에 관한 사무 중 국채 발행에 따라 수입되는 ...,제1항에 따라 처리하는 국채에 관한 사무 중 국채 발행에 따라 수입되는 자금과 국채...,"[{'severity': 'major', 'start_idx': 8, 'text':..."
78004,통화ㆍ국채ㆍ금융,국채법,The Governor of the Bank of Korea shall file a...,한국은행 총재는 제15조에 따른 국채에 관한 사무의 처리 내용을 기획재정부령으로 정...,총재는 제15조에 따른 국채에 관한 사무의 처리 내용을 기획재정부령으로 정하는 바에...,"[{'severity': 'major', 'start_idx': 0, 'text':..."


## 3.2.Minor - Omission (매우를 생략해서 의미가 약간 변질되는 경우)

In [ ]:
rindex1 = random_df(df_samp.loc[(df_samp.errors.apply(len) == 0)],0.2).index

In [ ]:
tag_name = "NNB"
set(count_tokens[rindex1].apply(lambda x: x[tag_name] if tag_name in x.keys() else []).sum())

In [ ]:
a = "심신의 장애로 직무를 집행하는 것이 매우 곤란하게 된 경우"
a[:20]+a[23:]

'심신의 장애로 직무를 집행하는 것이 곤란하게 된 경우'

In [ ]:
very_idx = df_samp.loc[(df_samp.KOR.str.contains("매우")) & (df_samp.errors.apply(lambda x: len(x))==0)].index
for i in very_idx:
    cur_hyp = df_samp.loc[i, "curHyp"]
    start = cur_hyp.find("매우")
    new_hyp = cur_hyp[:start]+cur_hyp[start+3: ]
    df_samp.loc[i, "curHyp"] = new_hyp
    df_samp.loc[i, "errors"].append({"severity":"minor", "start_idx":start, "text":"매우", "type":"omission"})
# very_df.KOR.apply(lambda x: x.find("매우"))

In [ ]:
df_samp.loc[very_idx, :]

,category,lawName,ENG,KOR,curHyp,errors
396497,통화ㆍ국채ㆍ금융,금융위원회의 설치 등에 관한 법률,Where it has become significantly difficult fo...,심신의 장애로 직무를 집행하는 것이 매우 곤란하게 된 경우,심신의 장애로 직무를 집행하는 것이 곤란하게 된 경우,"[{'severity': 'minor', 'start_idx': 20, 'text'..."
396605,통화ㆍ국채ㆍ금융,금융위원회의 설치 등에 관한 법률,The Governor may request the reconsideration o...,원장은 조정위원회의 의결사항이 위법하거나 공익에 비추어 매우 부당하다고 판단하면 재...,원장은 조정위원회의 의결사항이 위법하거나 공익에 비추어 부당하다고 판단하면 재의를 ...,"[{'severity': 'minor', 'start_idx': 31, 'text'..."
404615,통화ㆍ국채ㆍ금융,신용정보의 이용 및 보호에 관한 법률 시행령,"""Ground prescribed by Presidential Decree"" in ...","법 제14조제1항 단서에서 ""대통령령으로 정하는 사유""란 신용정보회사, 본인신용정보...","법 제14조제1항 단서에서 ""대통령령으로 정하는 사유""란 신용정보회사, 본인신용정보...","[{'severity': 'minor', 'start_idx': 139, 'text..."
81406,통화ㆍ국채ㆍ금융,대부업 등의 등록 및 금융이용자 보호에 관한 법률,Where a reasonable ground exists to conclude t...,의견청취가 매우 어렵거나 명백히 불필요하다고 인정되는 상당한 이유가 있는 경우,의견청취가 어렵거나 명백히 불필요하다고 인정되는 상당한 이유가 있는 경우,"[{'severity': 'minor', 'start_idx': 6, 'text':..."
82739,통화ㆍ국채ㆍ금융,예금자보호법,"The term ""insolvency-threatened financial comp...","""부실우려금융회사""란 재무구조가 취약하여 부실금융회사가 될 가능성이 매우 크다고 제...","""부실우려금융회사""란 재무구조가 취약하여 부실금융회사가 될 가능성이 크다고 제8조에...","[{'severity': 'minor', 'start_idx': 38, 'text'..."
82934,통화ㆍ국채ㆍ금융,예금자보호법,Before the Government conducts gratuitous tran...,정부는 제1항에 따른 무상 양여를 하기 전에 국무회의의 심의 및 대통령의 승인을 거...,정부는 제1항에 따른 무상 양여를 하기 전에 국무회의의 심의 및 대통령의 승인을 거...,"[{'severity': 'minor', 'start_idx': 90, 'text'..."
422286,통화ㆍ국채ㆍ금융,한국주택금융공사법,Where he or she is unable to perform his or he...,심신의 장애로 직무를 수행하기가 매우 어렵게 된 경우,심신의 장애로 직무를 수행하기가 어렵게 된 경우,"[{'severity': 'minor', 'start_idx': 18, 'text'..."
105888,상업ㆍ무역ㆍ공업,무역보험법,Where it has become impracticable or impossibl...,심신장애로 직무를 수행하는 것이 매우 곤란하게 되거나 불가능하게 된 경우,심신장애로 직무를 수행하는 것이 곤란하게 되거나 불가능하게 된 경우,"[{'severity': 'minor', 'start_idx': 18, 'text'..."


## 3.3.Neutral - Mis-translation(사소한 번역 오류)

In [ ]:
df_samp.loc[df_samp.curHyp.str.contains(r"달러|제곱미터|퍼센트")][["ENG", "curHyp"]]

,ENG,curHyp
394528,The amount of foreign investment to be investe...,관광사업에 투자하려는 외국인투자 금액이 미합중국화폐 5억달러 이상일 것
399090,The fees for guarantee which the management in...,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...
399094,The penalties for breach of contract which the...,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...
77386,Where a development project under subparagraph...,"법 제6조제1항제3호의2에 따른 단위개발사업지구(이하 ""단위개발사업지구""라 한다) ..."
77583,"Notwithstanding paragraphs (1) and (2), the si...",제1항 및 제2항에도 불구하고 「주택법」 제2조제6호에 따른 국민주택 규모 이하 주...
77650,If the State and a local government sell any S...,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...
77676,The price in an ordinary transaction in any fr...,법 제21조에 따라 경제자유구역에서 건당 10만 달러 이하의 경상거래에 따른 대가는...
77741,Each person who has been granted permission to...,카지노업 허가를 받은 자는 영업개시 후 2년이 지난 날부터 30일 이내에 제20조의...
408573,The limit of payment or receipt of amount that...,소액해외송금업자가 취급할 수 있는 건당 지급 및 수령 범위는 각각 미화 5천달러를 ...
408782,Where a resident makes a direct payment of cos...,거주자가 건당 미화 1만달러 이하의 경상거래에 따른 대가를 외국환업무취급기관등을 통...


In [ ]:
dollor_index = df_samp.loc[df_samp.curHyp.str.contains(r"미[^☞]+달러")].index

d1, d2 = "미합중국화폐 ", "미화 "
for i in dollor_index:
    cur_hyp = df_samp.loc[i, "curHyp"]
    end=cur_hyp.find("달러")
    start = cur_hyp.find(d1)
    if start==-1:
        start = cur_hyp.find(d2)
        new_hyp = cur_hyp.replace(d2, "")
        new_hyp = new_hyp.replace("달러", " USD")
    else:
        new_hyp = cur_hyp.replace(d1, "")
        new_hyp = new_hyp.replace("달러", " USD")
    df_samp.loc[i, "curHyp"] = new_hyp
    df_samp.loc[i, "errors"].append({"severity":"neutral", "start_idx":start, "text":cur_hyp[start:end+2], "type":"mis-translation"})

In [ ]:
df_samp.loc[dollor_index].iloc[:, 3:]

,KOR,curHyp,errors
394528,경제자유구역에서의 관광사업에 투자하려는 외국인투자 금액이 미합중국화폐 5억달러 이상일 것,관광사업에 투자하려는 외국인투자 금액이 5억 USD 이상일 것,"[{'severity': 'major', 'start_idx': 0, 'text':..."
77741,카지노업 허가를 받은 자는 영업개시 후 2년이 지난 날부터 30일 이내에 제20조의...,카지노업 허가를 받은 자는 영업개시 후 2년이 지난 날부터 30일 이내에 제20조의...,"[{'severity': 'major', 'start_idx': 58, 'text'..."
408573,소액해외송금업자가 취급할 수 있는 건당 지급 및 수령 범위는 각각 미화 5천달러를 ...,소액해외송금업자가 취급할 수 있는 건당 지급 및 수령 범위는 각각 5천 USD를 한...,"[{'severity': 'neutral', 'start_idx': 37, 'tex..."
408782,거주자가 건당 미화 1만달러 이하의 경상거래에 따른 대가를 외국환업무취급기관등을 통...,거주자가 건당 1만 USD 이하의 경상거래에 따른 대가를 외국환업무취급기관등을 통하...,"[{'severity': 'neutral', 'start_idx': 8, 'text..."
408809,법 제20조제1항에 따라 비거주자에 대한 채권보유현황을 보고하여야 하는 대상은 미화...,법 제20조제1항에 따라 비거주자에 대한 채권보유현황을 보고하여야 하는 대상은 1만...,"[{'severity': 'neutral', 'start_idx': 44, 'tex..."
408915,"법 제29조제1항제4호에서 ""대통령령으로 정하는 금액""이란 미화 3만달러를 말한다.","법 제29조제1항제4호에서 ""대통령령으로 정하는 금액""이란 3만 USD를 말한다.","[{'severity': 'neutral', 'start_idx': 33, 'tex..."
417058,미화 100만달러 이상으로서 금융위원회가 정하여 고시하는 금액 이상의 자기자본을 갖출 것,100만 USD 이상으로서 금융위원회가 정하여 고시하는 금액 이상의 자기자본을 갖출 것,"[{'severity': 'neutral', 'start_idx': 0, 'text..."
417060,직전 사업연도말 현재 운용자산규모(금융위원회가 정하여 고시하는 방법에 따라 계산한 ...,직전 사업연도말 현재 운용자산규모(금융위원회가 정하여 고시하는 방법에 따라 계산한 ...,"[{'severity': 'neutral', 'start_idx': 55, 'tex..."
408781,계약 건당 미화 5만달러 이내의 수출대금을 기획재정부장관이 정하여 고시하는 기간을 ...,계약 건당 5만 USD 이내의 수출대금을 기획재정부장관이 정하여 고시하는 기간을 초...,"[{'severity': 'neutral', 'start_idx': 6, 'text..."


In [ ]:
percent_idx = df_samp.loc[df_samp.curHyp.str.contains("퍼센트")].iloc[:,3:].index

In [ ]:
df_samp.loc[df_samp.curHyp.str.contains("퍼센트")].iloc[:,3:]

,KOR,curHyp,errors
399090,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...,[]
399094,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...,[]
77650,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...,"[{'severity': 'major', 'start_idx': 47, 'text'..."
519213,1세대무주택자의 경우: 보증금담보대출등을 받아 임차한 임차주택의 보증금 및 월세금액...,1세대무주택자의 경우: 보증금담보대출등을 받아 임차한 임차주택의 보증금 및 월세금액...,[]
519215,1세대무주택자의 경우: 보증금담보대출등 금액의 합산액(상환한 금액은 제외한다)의 3...,1세대무주택자의 경우: 보증금담보대출등 금액의 합산액(상환한 금액은 제외한다)의 3...,[]


In [ ]:
for i in percent_idx:
    cur_hyp = df_samp.loc[i, "curHyp"]
    start = cur_hyp.find("퍼센트")
    new_hyp = cur_hyp.replace("퍼센트", "%")
    df_samp.loc[i, "curHyp"] = new_hyp
    df_samp.loc[i, "errors"].append({"severity":"neutral", "start_idx":start, "text":"퍼센트", "type":"mis-translation"})
df_samp.loc[percent_idx]

,category,lawName,ENG,KOR,curHyp,errors
399090,통화ㆍ국채ㆍ금융,농림수산업자 신용보증법 시행령,The fees for guarantee which the management in...,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...,"[{'severity': 'neutral', 'start_idx': 75, 'tex..."
399094,통화ㆍ국채ㆍ금융,농림수산업자 신용보증법 시행령,The penalties for breach of contract which the...,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...,"[{'severity': 'neutral', 'start_idx': 58, 'tex..."
77650,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법 시행령,If the State and a local government sell any S...,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...,"[{'severity': 'major', 'start_idx': 47, 'text'..."
519213,사회복지,국민건강보험법 시행령,In cases of a member of a household without ho...,1세대무주택자의 경우: 보증금담보대출등을 받아 임차한 임차주택의 보증금 및 월세금액...,1세대무주택자의 경우: 보증금담보대출등을 받아 임차한 임차주택의 보증금 및 월세금액...,"[{'severity': 'neutral', 'start_idx': 109, 'te..."
519215,사회복지,국민건강보험법 시행령,In cases of a member of a household without ho...,1세대무주택자의 경우: 보증금담보대출등 금액의 합산액(상환한 금액은 제외한다)의 3...,1세대무주택자의 경우: 보증금담보대출등 금액의 합산액(상환한 금액은 제외한다)의 3...,"[{'severity': 'neutral', 'start_idx': 47, 'tex..."


In [ ]:
df_samp.loc[df_samp.errors.apply(lambda x: len(x))==0].shape[0]

18576

##3.4.Major - Omission (법령명을 생략하는 경우)

In [ ]:
law_index = df_samp.loc[(df_samp.errors.apply(lambda x: len(x))==0)&(df_samp.KOR.str.contains("「"))].index

In [ ]:
for i in law_index:
    cur_hyp = df_samp.loc[i, "curHyp"]
    start = cur_hyp.find("「")
    end = cur_hyp.find("」")
    addition_str = ""
    try:
        while cur_hyp[end+1]!=" ":
            addition_str += cur_hyp[end+1]
            end+=1
    except:
        pass

    df_samp.loc[i, "curHyp"] = cur_hyp[:start].rstrip()+" "+cur_hyp[end+1:].lstrip()
    df_samp.loc[i, "errors"].append({"severity":"major", "start_idx":start, "text":cur_hyp[start:end+1].strip(), "type":"omission"})

In [ ]:
df_samp.loc[law_index]

,category,lawName,ENG,KOR,curHyp,errors
547121,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the average monthly remuneration of each...,"예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술...","예술인[ 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술인""이라 한다...","[{'severity': 'major', 'start_idx': 4, 'text':..."
547123,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the amount of monthly remuneration of ea...,"노무제공자[「고용보험법」 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""...","노무제공자[ 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""단기노무제공자...","[{'severity': 'major', 'start_idx': 6, 'text':..."
394158,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"The term ""alien"" means a person who falls unde...","""외국인""이란 「외국인투자 촉진법」 제2조제1항제1호에 해당하는 자를 말한다.","""외국인""이란 제2조제1항제1호에 해당하는 자를 말한다.","[{'severity': 'major', 'start_idx': 8, 'text':..."
394159,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"The term ""foreign-invested enterprise"" means a...","""외국인투자기업""이란 「외국인투자 촉진법」 제2조제1항제6호에 따른 기업을 말한다.","""외국인투자기업""이란 제2조제1항제6호에 따른 기업을 말한다.","[{'severity': 'major', 'start_idx': 12, 'text'..."
394161,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"The term ""repatriating enterprise"" means an en...","""국내복귀기업""이란 「해외진출기업의 국내복귀 지원에 관한 법률」 제7조에 따라 지원...","""국내복귀기업""이란 제7조에 따라 지원대상으로 선정된 기업을 말한다.","[{'severity': 'major', 'start_idx': 11, 'text'..."
...,...,...,...,...,...,...
194486,노동,산업재해보상보험법 시행령,Software engineers defined in subparagraph 10 ...,「소프트웨어 진흥법」 제2조제3호의 소프트웨어사업에서 노무를 제공하는 같은 조 제1...,제2조제3호의 소프트웨어사업에서 노무를 제공하는 같은 조 제10호에 따른 소프트웨...,"[{'severity': 'major', 'start_idx': 0, 'text':..."
194490,노동,산업재해보상보험법 시행령,Where the employer submits documents required ...,「관세법」 제222조제1항 및 같은 법 시행령 제231조에 따라 세관장에게 보세운송...,제222조제1항 및 같은 법 시행령 제231조에 따라 세관장에게 보세운송업자 등록...,"[{'severity': 'major', 'start_idx': 0, 'text':..."
194491,노동,산업재해보상보험법 시행령,Where the employer enters information on the o...,「물류정책기본법」 제29조의2제5항에 따라 위험물질운송차량의 소유자 정보 및 운전자...,제29조의2제5항에 따라 위험물질운송차량의 소유자 정보 및 운전자 정보 등을 위험...,"[{'severity': 'major', 'start_idx': 0, 'text':..."
194492,노동,산업재해보상보험법 시행령,Where the employer submits a plan to transport...,「화학물질관리법」제15조제3항에 따라 환경부장관에게 유해화학물질 운반계획서를 제출한 경우,따라 환경부장관에게 유해화학물질 운반계획서를 제출한 경우,"[{'severity': 'major', 'start_idx': 0, 'text':..."


In [ ]:
df_samp.loc[(df_samp.errors.apply(lambda x: len(x))==0)].iloc[:, 2:4]

,ENG,KOR
547124,"""Where any ground prescribed by Presidential D...","법 제3조제1항에서 ""대통령령으로 정하는 사유에 해당하는 경우""란 다음 각 호의 어..."
547125,The standard remuneration under Article 3 (2) ...,법 제3조제2항(법 제48조의2제8항제2호 및 제48조의3제8항제2호에서 준용하는 ...
547126,Where an artist falls under any of the followi...,예술인이 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의2제8항제2호에서 준...
547127,Where a worker falls under any of the followin...,노무제공자가 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의3제8항제2호에서...
394152,The purpose of this Act is to facilitate forei...,이 법은 경제자유구역의 지정 및 운영을 통하여 외국인투자기업 및 국내복귀기업의 경영...
...,...,...
78001,The right to receive payments of principal and...,국채의 원금 및 이자를 받을 권리는 5년간 행사하지 아니하면 시효(시효)의 완성으로...
78006,Any person requested to provide materials rela...,제1항에 따라 국채 관련 자료의 제출을 요청받은 자는 특별한 사유가 없으면 요청에 ...
78007,Where necessary to formulate and carry out pol...,기획재정부장관은 국채시장에 관한 정책의 수립·운영에 필요한 경우에는 관계 중앙행정기...
406361,"Notwithstanding paragraph (3), a workplace cre...",제3항의 규정에 불구하고 제12조제1항제2호의 직장조합 및 동항제3호 라목의 단체조...


## 나중에

In [ ]:
pat = r"\"[ㄱ-ㅎ|가-힣]+\""
quote_comp = re.compile(pat)
quote_words = df_samp.loc[df_samp.KOR.str.contains(pat)].KOR.apply(lambda x: quote_comp.findall(x))
quote_words

547121                    ["단기예술인"]
547123                  ["단기노무제공자"]
394156                   ["경제자유구역"]
394158                      ["외국인"]
394159                  ["외국인투자기업"]
                    ...            
194419    ["위원장", "재심사위원회", "소위원회"]
194455             ["진찰", "특진의료기관"]
416399        ["의결권권유자", "의결권피권유자"]
194470             ["근로자", "진폐근로자"]
194496         ["근로자", "특수형태근로종사자"]
Name: KOR, Length: 1716, dtype: object

In [ ]:
quword4 = quote_words[quote_words.apply(lambda x: max(list(map(lambda k: len(k), x)))) > 8]

In [ ]:
qw_srs = quword4.apply(lambda x: sorted(x, key=lambda k: len(k), reverse=True)[0].strip('"').rstrip("등"))
sort_index = qw_srs.apply(len).sort_values().index
qw_srs[sort_index]

548555                출산전후급여
81365                 대부중개업자
397510                은행지주회사
207755                지정의료기관
83487                 여신금융기관
                 ...        
422493          주택담보노후연금보증채무
418500         금융투자상품거래청산업인가
416776        외국금융투자상품거래청산회사
556053        산업재해보상보험재심사위원회
519095    부당이득징수금체납정보공개심의위원회
Name: KOR, Length: 409, dtype: object

In [ ]:
origin_text = list(set(list(qw_srs.values)))
origin_text
# origin_text

## 다른 오류 생성

In [ ]:
df_samp.loc[df_samp.errors.apply(len)==0].shape[0]

3903

In [ ]:
not_yet_index = df_samp.loc[df_samp.errors.apply(len)==0].index

In [ ]:
temp = df_samp.loc[not_yet_index]
temp.loc[temp.curHyp.str.contains("할 수 있다")].iloc[:, 2:4]

In [ ]:
pattern = r"[\d]+조[^\항]*[\d]+항"
hangho = re.compile(pattern)
txtx = "집합투자업자는 제80조제4항 및 제86조제1항에도 불구하고 법 제81조제1항 단서 및 제234조제4항에 따라 상장지수집합투자기구(투자자 보호 등을 고려하여 금융위원회가 정하여 고시하는 상장지수집합투자기구에 한정한다)의 집합투자재산을 다음 각 호의 방법으로 운용할 수 있다."
sres = hangho.search(txtx)
print(sres)
fnum = txtx[sres.span()[0]]
snum = txtx[sres.span()[1]-2]
print(fnum, snum)

<re.Match object; span=(9, 15), match='80조제4항'>
8 4


In [ ]:
def make_dict(df):
    output_json = dict()
    output_json["data"] = []
    for i in df.index:
        base_form = {"category": None, "en": None, "errs": None, "hyp": None, "ko": None, "law_name": None}
        if len(df.loc[i, "errors"])==0:
            continue
        base_form["category"] = df.loc[i, "category"]
        base_form["en"] = df.loc[i, "ENG"]
        base_form["errs"] = df.loc[i, "errors"]
        base_form["hyp"] = df.loc[i, "curHyp"]
        base_form["ko"] = df.loc[i, "KOR"]
        base_form["law_name"] = df.loc[i, "lawName"]
        output_json["data"].append(base_form)
    return output_json

rrrr = make_dict(df_samp)

In [ ]:
with open(os.path.join(gdrive_path, "data_0823.json"), "w") as f:
  json.dump(rrrr, f, ensure_ascii=False, indent=4)